In [ ]:
from keras.models import load_model
import cv2
import numpy as np
import serial
import time

# 시리얼 포트 설정
ser = serial.Serial('COM2', 9600, timeout=1)
time.sleep(2)  # 아두이노가 리셋된 후 안정될 시간을 줍니다.

# 모델과 클래스 이름 로드
model = load_model("keras_Model.h5", compile=False)
class_names = [line.strip() for line in open("labels.txt", "r").readlines()]

# 웹캠 설정
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 224)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 224)

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
    image = np.asarray(frame, dtype=np.float32).reshape(1, 224, 224, 3)
    image = (image / 127.5) - 1

    # 예측
    prediction = model.predict(image)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    if confidence_score >= 0.7:
        label = class_name  # 라벨 값을 가져옴
        score = np.round(confidence_score * 100)
        text = f"{label}: {score}%"
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # 라벨 값에 따라 특정 문자 전송
        if label == 'can':
            ser.write(b'1')
        elif label == 'plastic':
            ser.write(b'2')
        elif label == 'none':
            ser.write(b'0')


    cv2.imshow("Webcam Image", frame)
    key = cv2.waitKey(1)

    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
ser.close()
